In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.models import Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation
import numpy as np 
from skimage.measure import compare_ssim
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import keras
import random
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras.utils import to_categorical
from math import log10, sqrt 
import cv2 
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

Using TensorFlow backend.


In [3]:
file_ = open('subset_cnn', 'rb')
original = pickle.load(file_)
adversarial = pickle.load(file_)
original_y = pickle.load(file_)
adversarial_y = pickle.load(file_)
file_.close()
threshold_psnr = 3.4

In [3]:
#file_ = open('subset_rbf', 'rb')
#original_rbf = pickle.load(file_)
#adversarial_rbf = pickle.load(file_)
#original_y_rbf = pickle.load(file_)
#adversarial_y_rbf = pickle.load(file_)
#file_.close()

In [4]:
inp = Input(shape=(28,28,1))
conv1 = Conv2D(32, (3,3),activation='relu')(inp)
conv2 = Conv2D(32, (3,3),activation='relu')(conv1)
max_pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
drop = Dropout(0.25)(max_pool1)
flat =  Flatten()(drop)
dense1 = Dense(128,activation="relu")(flat)
drop = Dropout(0.5)(dense1)
predictions = Dense(10,activation="softmax")(drop)
model = Model(inputs=[inp], outputs=[predictions])
model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.load_weights('CNN_MNIST.h5')
#print(model)
#model.summary()

(X, y), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X = X.reshape((X.shape[0], -1))
X_sc = X / 255.0
X_sc.shape
y_cat = to_categorical(y)
X_test = X_test.reshape((X_test.shape[0], -1))
X_test_sc = X_test / 255.0

num_lm0 = 100
num_lm = num_lm0 * 10
init_list = []
for ii in range(10):
    init_wgt0 = X[y==ii]
    init_wgt0 = init_wgt0[np.random.choice(range(init_wgt0.shape[0]), size=num_lm0, replace=False)] + \
                np.random.normal(scale=0.01, size=num_lm0*784).reshape(num_lm0, 784)
    init_list.append(init_wgt0)
init_wgt = np.vstack(init_list)
init_wgt = init_wgt[np.random.permutation(range(init_wgt.shape[0]))]

inp2 = Input(shape=(28*28,), name='inp')
oup2 = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp2)
oup2 = Dense(10, activation='softmax')(oup2)
model2 = Model(inp2, oup2)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.load_weights("rbfnn_v2.h5")

In [36]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_0_2k', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_0_2k', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)
        
        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.5079365079365079
accuracy on adversarials evolved against rbf: 0.9523809523809523
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8095238095238095
accuracy on adversarials evolved against rbf: 0.5158730158730159


In [44]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_1_2k', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_1_2k', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)

        
        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))

accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.49206349206349204
accuracy on adversarials evolved against rbf: 0.9603174603174603
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8253968253968254
accuracy on adversarials evolved against rbf: 0.5079365079365079


In [38]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_0_un', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_0_un', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)

        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.49206349206349204
accuracy on adversarials evolved against rbf: 0.9523809523809523
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8253968253968254
accuracy on adversarials evolved against rbf: 0.46825396825396826


In [39]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_1_un', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_1_un', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)

        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))
        


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.5158730158730159
accuracy on adversarials evolved against rbf: 0.9285714285714286
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8253968253968254
accuracy on adversarials evolved against rbf: 0.42857142857142855


In [40]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_0_qua', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_0_qua', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)
        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))
        


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.5079365079365079
accuracy on adversarials evolved against rbf: 0.9761904761904762
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8174603174603174
accuracy on adversarials evolved against rbf: 0.5


In [41]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_1_qua', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_1_qua', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)
        
        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.49206349206349204
accuracy on adversarials evolved against rbf: 0.9444444444444444
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8333333333333334
accuracy on adversarials evolved against rbf: 0.46825396825396826


In [42]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_0_mul', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_0_mul', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)

        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))
        


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.48412698412698413
accuracy on adversarials evolved against rbf: 0.9761904761904762
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8174603174603174
accuracy on adversarials evolved against rbf: 0.49206349206349204


In [43]:
psnrs_cnn_re = []
psnrs_rbf_re = []
psnrs_rbf = []
psnrs_cnn = []

file_ = open('adv_cnn_1_mul', 'rb')
adversarial_cnn = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

file_ = open('adv_rbf_1_mul', 'rb')
adversarial_rbf = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

for i in range(adversarial.shape[0]):
    psnrs_rbf.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_rbf[i].reshape(28,28)[5:-5,5:-5]))
    psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(28,28)[5:-5,5:-5], adversarial_cnn[i].reshape(28,28)[5:-5,5:-5]))
    
    
for i in range(adversarial.shape[0]):
        psnr_ = cv2.PSNR(adversarial[i].reshape(28,28),adversarial[i].reshape(28,28))
        psnr_cnn = round((psnrs_cnn[i]/psnr_)*100,2)
        psnr_rbf = round((psnrs_rbf[i]/psnr_)*100,2)

        if psnr_cnn > threshold_psnr:
            psnrs_cnn_re.append(adversarial_cnn[i])
        else:
            psnrs_cnn_re.append(adversarial[i])
            
        if psnr_rbf > threshold_psnr:
            psnrs_rbf_re.append(adversarial_rbf[i])
        else:
            psnrs_rbf_re.append(adversarial[i])     
            
psnrs_cnn_re = np.vstack(np.array(psnrs_cnn_re))
psnrs_rbf_re = np.vstack(np.array(psnrs_rbf_re))

model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


model_preds = np.argmax(model2.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_cnn_re).reshape(np.array(adversarial_cnn).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against cnn:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model2.predict(np.array(psnrs_rbf_re).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials evolved against rbf:', accuracy_score(model_preds, adversarial_y))


accuracy on subset adversarial: 0.9920634920634921
accuracy on adversarials evolved against cnn: 0.46825396825396826
accuracy on adversarials evolved against rbf: 0.9682539682539683
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials evolved against cnn: 0.8015873015873016
accuracy on adversarials evolved against rbf: 0.46825396825396826
